In [1]:
import numpy as np
from rank_bm25 import BM25Okapi
import pandas as pd

df_path = 'data/concated/'
articles = pd.read_csv(df_path + 'articles.csv', index_col="Unnamed: 0")
life_situations = pd.read_csv(df_path + 'life_situations.csv', index_col="Unnamed: 0")
news = pd.read_csv(df_path + 'news.csv', index_col="Unnamed: 0")
services = pd.read_csv(df_path + 'services.csv', index_col="Unnamed: 0")

In [2]:
article = articles[['true_id', 'text', 'url']]
life_situations.rename(columns={'URL': 'url'}, inplace=True)
life_situations = life_situations[['true_id', 'text', 'url']]
news = news[['true_id', 'text', 'url']]
services = services[['true_id', 'text', 'url']]

# Then you can concatenate all the dataframes into one
all_data = pd.concat([article, life_situations, news, services], ignore_index=True)
all_data

,true_id,text,url
0,2884,Бешенство – смертельная угроза для человека! Б...,https://www.gov.kz/memleket/entities/departame...
1,2885,Порядок администрирования налога на транспортн...,https://www.gov.kz/memleket/entities/kgd-karag...
2,2886,Спорт Миссия: формирование единой государствен...,https://www.gov.kz/memleket/entities/turkestan...
3,2887,ҚАЗАҚСТАННЫҢ МЕМЛЕКЕТТІК ОРГАНДАРЫМЕН БАЙЛАНЫС...,https://www.gov.kz/memleket/entities/qazalem/p...
4,2888,2022 жылға жергілікті атқарушы органдармен сыб...,https://www.gov.kz/memleket/entities/kyzylorda...
...,...,...,...
201948,2835,Aqparat ózektendirý satysynda. Úshinshi tulǵal...,https://beta2.egov.kz/services/4149?lang=qq
201949,2867,Memlekettik qyzmet kórsetý erezhesi Qurmetti Q...,https://beta2.egov.kz/services/3570?lang=qq
201950,2868,Возмещение затрат на повышение компетенции раб...,https://beta2.egov.kz/services/4724?lang=ru
201951,2869,Возмещение затрат на повышение эффективности о...,https://beta2.egov.kz/services/4725?lang=ru


In [3]:
import pickle

id_dict = all_data.reset_index().set_index('index')['true_id'].to_dict()
with open('application/id_dict.pickle', 'wb') as handle:
    pickle.dump(id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

all_data.to_csv('application/all_data.csv')

In [4]:
corpus = all_data['text'].values
corpus

array(['Бешенство – смертельная угроза для человека! Бешенство – природно-очаговое особо опасное, смертельное заболевание. Среди инфекционных болезней бешенство занимает особое место в силу абсолютной летальности (если в первые же часы не оказать зараженному человеку необходимую помощь, он неминуемо погибнет). Бешенство является неизлечимой болезнью, смерть наступает от паралича дыхания и упадка сердечной деятельности. Человек умирает в полном сознании и в страшных муках. Северо-Казахстанская область находится в зоне природного очага бешенства. В нашей области эпизоотическая активность природных очагов отмечается в зимне-весенний период, когда опасный вирус чаще заносится дикими животными в населенные пункты, где заражаются собаки, кошки, сельскохозяйственные животные и возрастает риск заражения людей. Благодаря активной профилактической работе в Северо-Казахстанской области заболевания людей бешенством не отмечается с 1998 года. Естественным резервуаром являются грызуны. Домашние живо

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize
from tqdm import tqdm

def stem_token(token, stemmer):
    return stemmer.stem(token) if any(c.isalpha() for c in token) else ""

english_stemmer = PorterStemmer()
russian_stemmer = SnowballStemmer("russian")
stop_words = set(stopwords.words('english')).union(set(stopwords.words('russian')))

def preprocess(doc):
    doc = doc.lower().replace('\n', ' ')
    tokens = word_tokenize(doc)
    stemmed_tokens = [stem_token(token, english_stemmer) for token in tokens if token not in stop_words]
    result = [stem_token(token, russian_stemmer) for token in stemmed_tokens if token not in stop_words]

    return result

preprocessed_corpus = [preprocess(doc) for doc in tqdm(corpus)]

100%|██████████| 201953/201953 [51:17<00:00, 65.62it/s]  


In [46]:
with open('application/preprocessed_corpus.pickle', 'wb') as handle:
    pickle.dump(preprocessed_corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

bm25 = BM25Okapi(preprocessed_corpus, k1=2.3, b=0.8)
with open('application/bm25.pickle', 'wb') as handle:
    pickle.dump(bm25, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
test = pd.read_csv('epir_test.csv')
test

,id,question
0,1,Алғашқы медициналық-санитариялық көмек көрсете...
1,2,How can I make an online reservation for servi...
2,3,Online services Egov
3,4,Telegram bot Egov servces
4,5,Where can I find the latest updates and inform...
...,...,...
119,120,How to change your name in Kazakhstan?
120,121,"Requirements for changing surname, name, or pa..."
121,122,Process of changing name due to marriage or di...
122,123,Can I change my name online in Kazakhstan?


In [48]:
def get_indexes(query, n):
    preprocessed_query = preprocess(query)
    doc_scores = bm25.get_scores(preprocessed_query)
    indices = np.argsort(doc_scores)[-n:]  # get indices of top 5 scores
    relevant_docs = [id_dict[index] for index in reversed(indices)]  # reverse to start from highest
    return relevant_docs

def get_most_relevant_index(query, n=5):
    indexes = get_indexes(query, n)
    return indexes[0]

In [49]:
from tqdm import tqdm
tqdm.pandas()

# apply get_index, but save the most relevant document
test['index'] = test['question'].progress_apply(get_most_relevant_index)
test[['id', 'index']].to_csv('submission.csv', index=False)

  0%|          | 0/124 [00:00<?, ?it/s]

100%|██████████| 124/124 [01:06<00:00,  1.86it/s]
